In [18]:
from openai import OpenAI


client=OpenAI(api_key='')

In [19]:
training_file_small_all ='Data/lat/training_small_combined_conversational.jsonl'
validation_file_small_all = 'Data/lat/validation_small_combined_conversational.jsonl'

In [20]:
training_file_small_all=client.files.create(file=open(training_file_small_all,'rb'),
                    purpose='fine-tune')

validation_file_small_all=client.files.create(file=open(validation_file_small_all,'rb'),
                    purpose='fine-tune')
print(training_file_small_all)
print('---------------------------------')
print(validation_file_small_all)

FileObject(id='file-2NyDtDWkUquQzN2STSSqct', bytes=2074238, created_at=1744298505, filename='training_small_combined_conversational.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)
---------------------------------
FileObject(id='file-ByHuXxJEayPLcRNmiE7WnA', bytes=231982, created_at=1744298506, filename='validation_small_combined_conversational.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)


In [21]:
response_small_all=client.fine_tuning.jobs.create(
    training_file=training_file_small_all.id,
    validation_file=validation_file_small_all.id,
    model='gpt-4o-mini-2024-07-18',
    hyperparameters={
    'n_epochs':'auto',
    'batch_size':'auto',  
})

print(response_small_all)

FineTuningJob(id='ftjob-9rdJVUwuLvnSv0f7aEFD4YJL', created_at=1744298508, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-AVOdG0yGREZO41elnVjY6u7y', result_files=[], seed=766645144, status='validating_files', trained_tokens=None, training_file='file-2NyDtDWkUquQzN2STSSqct', validation_file='file-ByHuXxJEayPLcRNmiE7WnA', estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})


In [29]:
ft_model_small_all=client.fine_tuning.jobs.retrieve(response_small_all.id)#(response.id)
print(ft_model_small_all)

FineTuningJob(id='ftjob-9rdJVUwuLvnSv0f7aEFD4YJL', created_at=1744298508, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI', finished_at=1744300923, hyperparameters=Hyperparameters(n_epochs=3, batch_size=5, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-AVOdG0yGREZO41elnVjY6u7y', result_files=['file-2N78z8wM7npZDAJ944ntnE'], seed=766645144, status='succeeded', trained_tokens=1638468, training_file='file-2NyDtDWkUquQzN2STSSqct', validation_file='file-ByHuXxJEayPLcRNmiE7WnA', estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 5, 'learning_rate_multiplier': 1.8}}})


In [45]:
system_prompt = "You are a polymer expert specializing in thermoset shape memory polymers. Your role is to analyze and create monomer pairs with excellent thermal stability and mechanical strength."
user_message_1 =['I want to make a thermoset shape memory polymer','Please suggest me some TSMPs']
user_message_2 =['Please focus on property based monomer pairs','Please focus on group based monomer pairs']
proeprty_specific_message = ["Please give me some TSMP with Tg = 100C and Er= 150MPa","Please generate some TSMP with Tg = 50C and Er= 100Mpa"]
group_specific_message = ["Please give me some TSMP with epoxy(C1OC1) groups in monomer 1 and imine(NC) groups in monomer 2","Please generate some TSMP with vinyl(C=C) groups in monomer 1 and thiol(CCS) groups in monomer 2"]
mixed_specific_message = ["Please give me some TSMP with Tg = 100C and Er= 150MPa and vinyl(C=C) groups in monomer 1 and vinyl(C=C) groups in monomer 2","Please generate some TSMP with Tg = 50C and Er= 100Mpa and Thiol(CCS) groups in monomer 1 and vinyl(C=C) groups in monomer 2"]


In [34]:
messages=[]
messages.append({"role":"system","content":system_prompt})
def generate_new_TSMP(role,prompt_content, isFinalQuery=False):
    propmt={"role":role, "content": prompt_content}
    messages.append(propmt)
    if not isFinalQuery:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages
        )
        result = completion.choices[0].message.content
    else:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages,
            temperature=1.0,
            max_tokens=200,
            n=2
        )
        result= completion
       
        
   
    messages.append({"role":'assistant', "content": result})
    
    # Print the response from the assistant
    return result, messages


replies_0, messages_0 = generate_new_TSMP('user',user_message_1[0], isFinalQuery=False)
print("User: ",user_message_1[0])
print("Assistant: ",replies_0)
replies_1, messages_1 = generate_new_TSMP('user',user_message_2[0], isFinalQuery=False)
print("User: ",user_message_2[0])
print("Assistant: ",replies_1)
replies_2, messages_2 = generate_new_TSMP('user',proeprty_specific_message[0], isFinalQuery=True)
print("User: ",proeprty_specific_message[0])
print("Assistant: Two TSMPs for you ( as I told model to generate two samples per query) ")
for i in range(2):
    print("Sample : :",i+1, replies_2.choices[i].message.content)
    

User:  I want to make a thermoset shape memory polymer
Assistant:  What's your priority for the TSMP - working with specific groups or meeting certain property targets?
User:  Please focus on property based monomer pairs
Assistant:  I'll need two key properties to design your TSMP: the glass transition temperature (Tg) and stress recovery (Er). What values would you like?
User:  Please give me some TSMP with Tg = 100C and Er= 150MPa
Assistant: Two TSMPs for you ( as I told model to generate two samples per query) 
Sample : : 1 Great choice of properties and groups. Here's a compatible monomer pair:
Monomer 1: C=C(C)C(=O)OC
Monomer 2: C=C(C)C(=O)OCCOCCOCCOCCOCCOCCOCCOCCOCCOC(=O)C(=C)C
Sample : : 2 Based on your requirements, I suggest the following monomers:
Monomer 1: c3cc(N(CC1CO1)CC2CO2)ccc3OCC4CO4
Monomer 2: NCCNCCN(CCNCCC(CN)CCN)CCN(CCNCCN)CCN(CCN)CCN


In [37]:
messages=[]
messages.append({"role":"system","content":system_prompt})
def generate_new_TSMP(role,prompt_content, isFinalQuery=False):
    propmt={"role":role, "content": prompt_content}
    messages.append(propmt)
    if not isFinalQuery:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages
        )
        result = completion.choices[0].message.content
    else:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages,
            temperature=1.0,
            max_tokens=200,
            n=2
        )
        result= completion
       
        
   
    messages.append({"role":'assistant', "content": result})
    
    # Print the response from the assistant
    return result, messages

replies_0, messages_0 = generate_new_TSMP('user',user_message_1[1], isFinalQuery=False)
print("User: ",user_message_1[1])
print("Assistant: ",replies_0)
replies_1, messages_1 = generate_new_TSMP('user',user_message_2[1], isFinalQuery=False)
print("User: ",user_message_2[1])
print("Assistant: ",replies_1)
replies_2, messages_2 = generate_new_TSMP('user',group_specific_message[0], isFinalQuery=True)
print("User: ",group_specific_message[0])
print("Assistant: Two TSMPs for you ( as I told model to generate two samples per query) ")
for i in range(2):
    print("Sample : :",i+1, replies_2.choices[i].message.content)

User:  Please suggest me some TSMPs
Assistant:  Would you like to design your TSMP based on specific functional groups or target properties?
User:  Please focus on group based monomer pairs
Assistant:  Please specify your required functional groups - I need one for one monomer and another for the second.
User:  Please give me some TSMP with epoxy(C1OC1) groups in monomer 1 and imine(NC) groups in monomer 2
Assistant: Two TSMPs for you ( as I told model to generate two samples per query) 
Sample : : 1 Great choice of groups. Here's a compatible monomer pair:
Monomer 1: Cc3cc(c2cc(C)c(OCC1CO1)c(C)c2)cc(C)c3OCC4CO4
Monomer 2: CC1(C)CC(N)CC(C)(CN)C1
Sample : : 2 Based on your requirements, I suggest the following monomers:
Monomer 1 with C1OC1 groups: CC(C)(c2ccc(OC1CO1)cc2)c4ccc(OC3CO3)cc4
Monomer 2 with NC groups: NCCCOCCOCCCN


In [42]:
messages=[]
messages.append({"role":"system","content":system_prompt})
def generate_new_TSMP(role,prompt_content, isFinalQuery=False):
    propmt={"role":role, "content": prompt_content}
    messages.append(propmt)
    if not isFinalQuery:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages
        )
        result = completion.choices[0].message.content
    else:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages,
            temperature=1.0,
            max_tokens=200,
            n=2
        )
        result= completion
       
        
   
    messages.append({"role":'assistant', "content": result})
    
    # Print the response from the assistant
    return result, messages

replies, messages = generate_new_TSMP('user',group_specific_message[1], isFinalQuery=True)
print("User: ",group_specific_message[1])
print("Assistant: Two TSMPs for you ( as I told model to generate two samples per query) ")
for i in range(2):
    print("Sample : :",i+1, replies.choices[i].message.content)

User:  Please generate some TSMP with vinyl(C=C) groups in monomer 1 and thiol(CCS) groups in monomer 2
Assistant: Two TSMPs for you ( as I told model to generate two samples per query) 
Sample : : 1 These monomers match the required group profile:
- Monomer 1 has C=C groups: C=C(C)C(=O)OCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOC(=O)C(=C)C
- Monomer 2 has CCS groups: O=c1n(CCS)c(=O)n(CCS)c(=O)n1CCS
Sample : : 2 Here is a group-compliant monomer pair:
Monomer 1 includes C=C groups: C=C(C)C(=O)OCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOCCOC(=O)C(=C)C
Monomer 2 includes CCS groups: O=c1n(CCS)c(=O)n(CCS)c(=O)n1CCS


In [43]:
messages=[]
messages.append({"role":"system","content":system_prompt})
def generate_new_TSMP(role,prompt_content, isFinalQuery=False):
    propmt={"role":role, "content": prompt_content}
    messages.append(propmt)
    if not isFinalQuery:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages
        )
        result = completion.choices[0].message.content
    else:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages,
            temperature=1.0,
            max_tokens=200,
            n=2
        )
        result= completion
       
        
   
    messages.append({"role":'assistant', "content": result})
    
    # Print the response from the assistant
    return result, messages

replies, messages = generate_new_TSMP('user',proeprty_specific_message[1], isFinalQuery=True)
print("User: ",proeprty_specific_message[1])
print("Assistant: Two TSMPs for you ( as I told model to generate two samples per query) ")
for i in range(2):
    print("Sample : :",i+1, replies.choices[i].message.content)

User:  Please generate some TSMP with Tg = 50C and Er= 100Mpa
Assistant: Two TSMPs for you ( as I told model to generate two samples per query) 
Sample : : 1 Here is a TSMP designed for Tg = 50C and Er = 100Mpa:
Monomer 1: CC(C)(c2ccc(OCC1CO1)cc2)c4ccc(OCC3CO3)cc4
Monomer 2: CC(N)COCC(C)COCC(C)COCC(C)COCC(C)COCC(C)COCC(C)N
Sample : : 2 To achieve Tg of 50C and Er of 100Mpa, consider this monomer combination:
Monomer 1: C=CCc1ccccc1OCC(O)COC(=O)Cc3ccc(C(=O)OCC(O)COc2ccccc2CC=C)cc3
Monomer 2: C=CCOCC(CO)(COCC=C)COCC=C


In [46]:
messages=[]
messages.append({"role":"system","content":system_prompt})
def generate_new_TSMP(role,prompt_content, isFinalQuery=False):
    propmt={"role":role, "content": prompt_content}
    messages.append(propmt)
    if not isFinalQuery:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages
        )
        result = completion.choices[0].message.content
    else:
        completion = client.chat.completions.create(
            model='ft:gpt-4o-mini-2024-07-18:personal::BKodpSOI',
            messages=messages,
            temperature=1.0,
            max_tokens=200,
            n=2
        )
        result= completion
       
        
   
    messages.append({"role":'assistant', "content": result})
    
    # Print the response from the assistant
    return result, messages

replies, messages = generate_new_TSMP('user',mixed_specific_message[0], isFinalQuery=True)
print("User: ",mixed_specific_message[1])
print("Assistant: Two TSMPs for you ( as I told model to generate two samples per query) ")
for i in range(2):
    print("Sample : :",i+1, replies.choices[i].message.content)

User:  Please generate some TSMP with Tg = 50C and Er= 100Mpa and Thiol(CCS) groups in monomer 1 and vinyl(C=C) groups in monomer 2
Assistant: Two TSMPs for you ( as I told model to generate two samples per query) 
Sample : : 1 To meet the requirements of Tg = 100C and Er = 150Mpa, with C=C groups in monomers 1 and 2, I suggest this combination:
Monomer 1: C=C(C)C(=C)OCOCCOC(=O)C(=C)C
Monomer 2: C=C(C)C(=O)OCCOCCOCCOCCOCCOCCOCCOCCOCCOC(=O)C(=C)C
Sample : : 2 To achieve Tg of 100C and Er of 150Mpa with monomers containing C=C groups, try:
Monomer 1: C=C(C)C(=O)OCCOCCOCCOCCOCCOCCOCCOCCOC(=O)C(=C)C
Monomer 2: C=C(C)C(=O)OCCOCCOCCOCCOCCOC(=O)C(C)=O
